In [1]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score
from itertools import cycle, islice
from operator import itemgetter
import sif_embedding_wrapper
import pandas as pd
import numpy as np
import itertools
import codecs
import utils
import os

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

# model = KeyedVectors.load_word2vec_format('/home/stirunag/models/model_OTAR_200d-3mc-10it.bin', binary=True)
# model.save_word2vec_format('/home/stirunag/pre-trained_word_embeddings/model_OTAR_200d-3mc-10it.txt', binary=False)

# sed -i -n -e '2,$p'file.txt to remove first line from the txt file


# words, embs, weight4ind = sif_embedding_wrapper.load_embeddings("/home/stirunag/pre-trained_word_embeddings/model_OTAR_200d-3mc-10it.txt", 
#                                                      '/home/stirunag/pre-trained_word_embeddings/OTAR/TF.txt')


# words, embs, weight4ind = sif_embedding_wrapper.load_embeddings("/home/stirunag/pre-trained_word_embeddings/PubMed-and-PMC-FS.txt", 
#                                                      '/home/stirunag/pre-trained_word_embeddings/wiki/enwiki_vocab_min200.txt')

words, embs, weight4ind = sif_embedding_wrapper.load_embeddings("/home/stirunag/pre-trained_word_embeddings/glove/glove.6B.300d.txt", 
                                                      '/home/stirunag/pre-trained_word_embeddings/wiki/enwiki_vocab_min200.txt')

In [ ]:
# get the current working directory
data_path = os.path.abspath(os.path.join(os.path.dirname( '__file__' ), '..', 'Datasets'))+'/'

# Although the dataset says csv, it is tab delimited. In addition to this, they have severe codels problems. 
# So best to parse throught codes first. 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 2: invalid start byte

#open for reading with "universal" type set

import codecs

doc_d_t = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_target'+'.csv','rU','UTF-8') 
EUADR_drug_target = pd.read_csv(doc_d_t, sep='\t', na_filter = False)
EUADR_drug_target['CLASS'] = 'drug_gene'

doc_t_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_target_disease'+'.csv','rU','UTF-8',errors='ignore') 
EUADR_target_disease = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
EUADR_target_disease['CLASS'] = 'gene_disease'
       
doc_d_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_disease'+'.csv','rU','UTF-8')                       
EUADR_drug_disease = pd.read_csv(doc_d_d, sep='\t', na_filter = False)
EUADR_drug_disease['CLASS'] = 'drug_disease'


EUADR_temp = EUADR_drug_target.append(EUADR_target_disease).append(EUADR_drug_disease)

EUADR_temp_1 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'PA']
EUADR_temp_2 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'NA']
EUADR_temp = EUADR_temp_1.append(EUADR_temp_2)

EUADR_temp.reset_index(inplace=True)

In [ ]:
# Get GAD dataset

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_Y_N'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_Y_N = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_Y_N['CLASS'] = 'gene_disease'

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_F'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_F = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_F['CLASS'] = 'gene_disease'

GAD_temp = GAD_target_disease_Y_N


In [ ]:
EUADR_temp.reset_index(inplace=True)

doc_embeddings = sif_embedding_wrapper.sentences2vecs(EUADR_temp["SENTENCE"], embs, words, weight4ind)
EUADR_temp["vector"] = pd.Series(list(doc_embeddings))

In [ ]:
EUADR_temp.columns

In [ ]:
ground_truth = {}

for idx, row in EUADR_temp.iterrows():
    ground_truth[idx] = row['ASSOCIATION_TYPE']

categories = list(EUADR_temp["ASSOCIATION_TYPE"].unique())
categories    

In [ ]:
ground_truth = {}
for idx, row in EUADR_temp.iterrows():
    ground_truth[idx] = row['ASSOCIATION_TYPE'] 

inv_map = {}
for k, v in ground_truth.items():
    inv_map[v] = inv_map.get(v, [])
    inv_map[v].append(k)

# inv_map

In [ ]:
# Get average/mean of the sentence vectors that represent our topics 
category_vecs = {}
for c in categories:
    vectors = np.asarray(list(EUADR_temp.loc[EUADR_temp.index.isin(inv_map[c])].vector))
    category_vecs[c] = np.mean(vectors, axis=0)

    
# category_vecs    

In [ ]:
# Try to predict the label of unknown sentences

predictions = {}

selected_idx = [j for i in inv_map.values() for j in i]

for idx, row in EUADR_temp.iterrows():
    if idx in selected_idx:
        max_sim = 0
        winner = 'other'
        for j in category_vecs:
            sim = cosine_similarity(row["vector"].reshape(1, -1), category_vecs[j].reshape(1, -1)).flatten()[0]
            if sim > max_sim:
                max_sim = sim
                winner = j
        predictions[idx] = winner
    

In [ ]:
def get_accuracy_score(predictions, truth_dict):
    preds = []
    labels = []
    mis_classified = []
    mis_pred = []
    
    for k,v in predictions.items():
        preds.append(v)
        labels.append(truth_dict[k])
        if v!=truth_dict[k]:
#             print(str(v) + '--x--' + str(truth_dict[k]))
            mis_pred.append(str(v))
            mis_classified.append(k)

    return f1_score(labels, preds, average='weighted'), mis_classified, mis_pred


score, miss_classified_df, miss_pred = get_accuracy_score(predictions, ground_truth)
score

In [ ]:
pd.set_option('display.max_colwidth', -1)

miss_calssified_df = EUADR_temp.iloc[miss_classified_df , [13,2]] 
miss_calssified_df['Predicted-CLASS'] = miss_pred
miss_calssified_df

# result_path = os.path.abspath(os.path.join(os.path.dirname( '__file__' ), '..', 'Results'))+'/'
# miss_calssified_df.to_csv(result_path+'miss_predictions_train.csv')

In [ ]:
GAD_temp.reset_index(inplace=True)

doc_embeddings = sif_embedding_wrapper.sentences2vecs(GAD_temp["GAD_CONCLUSION"], embs, words, weight4ind)
GAD_temp["vector"] = pd.Series(list(doc_embeddings))

GAD_temp['GAD_ASSOC'] = GAD_temp['GAD_ASSOC'].apply(lambda x: x.replace('Y', 'PA').replace('N', 'NA').replace('F', 'FA'))
GAD_temp.columns

In [ ]:
GAD_test_ground_truth = {}

for idx, row in GAD_temp.iterrows():
    GAD_test_ground_truth[idx] = row['GAD_ASSOC']
    
# Try to predict the label of unknown sentences

GAD_test_predictions = {}

for idx, row in GAD_temp.iterrows():
    max_sim = 0.00
    winner = 'other'
    for j in category_vecs:
        sim = cosine_similarity(row["vector"].reshape(1, -1), category_vecs[j].reshape(1, -1)).flatten()[0]
        if sim > max_sim:
            max_sim = sim
            winner = j
    GAD_test_predictions[idx] = winner   
    
    
GAD_score, GAD_miss_classified, GAD_miss_pred = get_accuracy_score(GAD_test_predictions, GAD_test_ground_truth)
GAD_score    

In [ ]:
miss_calssified_GAD_test = GAD_temp.iloc[GAD_miss_classified , [11,2]] 
miss_calssified_GAD_test['Predicted-CLASS'] = GAD_miss_pred
miss_calssified_GAD_test
# miss_calssified_GAD_test.to_csv(result_path+'miss_predictions_test.csv')